Robot Localization with Python and Particle Filters
===================================================

Import libraries and load map.

In [34]:
import numpy as np
import cv2

map = cv2.imread(r"C:\Users\Zidan\Downloads\Files (3)\Files\home\jovyan\work\map.png", 0)
HEIGHT, WIDTH = map.shape
print(map)

rx, ry, rtheta = (WIDTH / 4, HEIGHT / 4, 0)

[[ 79  80  82 ... 133 148 156]
 [ 78  79  81 ... 138 156 164]
 [ 76  77  79 ... 147 170 180]
 ...
 [181 181 182 ... 174 172 171]
 [178 178 178 ... 180 179 178]
 [177 177 177 ... 183 182 182]]


Map coordinate system

![title](images/coords.png)

CAUTION: The terrain height at X,Y coordinates is map(Y,X).

Read keyboard input.

In [35]:
STEP = 5
TURN = np.radians(25)

def get_input():
  fwd = 0
  turn = 0
  halt = False
  k = cv2.waitKey(0)
  if k == 82: 
    fwd = STEP
  elif k == 83: 
    turn = TURN
  elif k == 81: 
    turn = -TURN
  elif k == 27:  # Esc key
    halt = True
  return fwd, turn, halt

Move the robot, with Gausssian noise.

![title](images/gaussian.png)

In [36]:
SIGMA_STEP = 0.5
SIGMA_TURN = np.radians(5)

def move_robot(rx, ry, rtheta, fwd, turn):
  fwd_noisy = np.random.normal(fwd, SIGMA_STEP, 1)
  rx += fwd_noisy * np.cos(rtheta)
  ry += fwd_noisy * np.sin(rtheta)
  print("fwd_noisy = ", fwd_noisy)
  
  turn_noisy = np.random.normal(turn, SIGMA_TURN, 1)
  rtheta += turn_noisy
  print("turn_noisy=", np.degrees(turn_noisy))
  
  return rx, ry, rtheta

Initialize particle cloud.

In [37]:
NUM_PARTICLES = 3000

def init():
    particles = np.random.rand(NUM_PARTICLES, 3)
    particles *= np.array( (WIDTH, HEIGHT, np.radians(360)) )
    return particles

Move the particles.

In [38]:
def move_particles(particles, fwd, turn):
    particles[:,0] += fwd * np.cos(particles[:,2])
    particles[:,1] += fwd * np.sin(particles[:,2])
    particles[:,2] += turn

    particles[:,0] = np.clip(particles[:,0], 0.0, WIDTH-1)
    particles[:,1] = np.clip(particles[:,1], 0.0, HEIGHT-1)
    return particles

Get value from robot's sensor.

In [39]:
SIGMA_SENSOR = 2
def sense(x, y, noisy=False):
    x = int(x)
    y = int(y)
    if noisy:
        return np.random.normal(map[y,x], SIGMA_SENSOR, 1)
    return map[y,x]

Compute particle weights.

In [40]:
def compute_weights(particles, robot_sensor):    
    errors = np.zeros(NUM_PARTICLES)
    for i in range(NUM_PARTICLES):
        particle_sensor = sense(particles[i,0], particles[i,1])
        errors[i] = abs(robot_sensor - particle_sensor)
    weights = np.max(errors) - errors

    # Kill off particles on edge
    weights[
        (particles[:,0] == 0) |
        (particles[:,0] == WIDTH-1) |
        (particles[:,1] == 0) |
        (particles[:,1] == HEIGHT-1)
    ] = 0.0

    # Increase sensitivity
    weights = weights ** 3
    return weights

Resample the particles.

In [41]:
def resample(particles, weights):
    probabilities = weights/np.sum(weights)
    new_index = np.random.choice(
        NUM_PARTICLES,
        size=NUM_PARTICLES,
        p = probabilities
    )

    particles = particles[new_index,:]
    return particles

Add noise to the particles.

In [42]:
SIGMA_POS = 2
SIGMA_TURN = np.radians(10)

def add_noise(particles):
    noise = np.concatenate(
        (
            np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
            np.random.normal(0, SIGMA_POS, (NUM_PARTICLES, 1)),
            np.random.normal(0, SIGMA_TURN, (NUM_PARTICLES, 1)),
        ),
        axis=1
    )
    particles += noise
    return particles

Display robot, particles and best guess.

In [43]:
def display(map, rx, ry, particles):
    lmap = cv2.cvtColor(map, cv2.COLOR_GRAY2BGR)
    
    # Display particles
    if len(particles) > 0:
        for i in range(NUM_PARTICLES):
            cv2.circle(lmap, 
                       (int(particles[i,0]), int(particles[i,1])), 
                       1, 
                       (255,0,0), 
                       1)
        
    # Display robot
    cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)

    # Display best guess
    if len(particles) > 0:
        px = np.mean(particles[:,0])
        py = np.mean(particles[:,1])
        cv2.circle(lmap, (int(px), int(py)), 5, (0,0,255), 5)

    cv2.imshow('map', lmap)

Main routine.

In [44]:
particles = init()
while True:
    display(map, rx, ry, particles)
    fwd, turn, halt = get_input()
    if halt:
        break
    rx, ry, rtheta = move_robot(rx, ry, rtheta, fwd, turn)
    particles = move_particles(particles, fwd, turn)
    if fwd != 0:
        robot_sensor = sense(rx, ry, noisy=True)        
        weights = compute_weights(particles, robot_sensor)
        particles = resample(particles, weights)
        particles = add_noise(particles)
    
cv2.destroyAllWindows()                        


fwd_noisy =  [-0.29193382]
turn_noisy= [27.50731354]
fwd_noisy =  [-0.11622965]
turn_noisy= [-29.54699029]
fwd_noisy =  [0.27344736]
turn_noisy= [12.46533661]
fwd_noisy =  [-0.32412066]
turn_noisy= [-40.93168405]
fwd_noisy =  [0.33178869]
turn_noisy= [18.73691178]
fwd_noisy =  [-0.88224831]
turn_noisy= [-10.3558908]
fwd_noisy =  [-0.1457903]
turn_noisy= [32.29120876]


C:\Users\Zidan\AppData\Local\Temp\ipykernel_3164\1949318807.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.circle(lmap, (int(rx), int(ry)), 5, (0,255,0), 10)


fwd_noisy =  [5.65591092]
turn_noisy= [7.90652783]


C:\Users\Zidan\AppData\Local\Temp\ipykernel_3164\1271927725.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x = int(x)
C:\Users\Zidan\AppData\Local\Temp\ipykernel_3164\1271927725.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y = int(y)
C:\Users\Zidan\AppData\Local\Temp\ipykernel_3164\381150853.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  errors[i] = abs(robot_sensor - particle_sensor)


fwd_noisy =  [5.80897288]
turn_noisy= [-6.25810152]
fwd_noisy =  [5.25642942]
turn_noisy= [-5.53846388]
fwd_noisy =  [5.47213074]
turn_noisy= [20.35216505]
fwd_noisy =  [5.46149905]
turn_noisy= [6.53369817]
fwd_noisy =  [4.85776344]
turn_noisy= [-6.70037069]
fwd_noisy =  [4.8173556]
turn_noisy= [-7.90128617]
fwd_noisy =  [4.64526148]
turn_noisy= [-4.35460351]
fwd_noisy =  [5.18366297]
turn_noisy= [-17.78900263]
fwd_noisy =  [4.42821666]
turn_noisy= [-16.21344537]
fwd_noisy =  [5.39257313]
turn_noisy= [-0.70359059]
fwd_noisy =  [4.37382355]
turn_noisy= [-1.80216524]
fwd_noisy =  [3.94999796]
turn_noisy= [1.48541315]
fwd_noisy =  [5.02323635]
turn_noisy= [3.15346662]
fwd_noisy =  [4.56243048]
turn_noisy= [11.57303873]
fwd_noisy =  [4.90405881]
turn_noisy= [2.69869143]
fwd_noisy =  [5.40517079]
turn_noisy= [2.41405863]
fwd_noisy =  [0.16487378]
turn_noisy= [15.12536974]
fwd_noisy =  [-0.302592]
turn_noisy= [19.76341045]
fwd_noisy =  [-0.03645808]
turn_noisy= [13.15812687]
fwd_noisy =  [0.